# BPI Challenge 2018

In [1]:
import pandas as pd
from datetime import datetime

## Understanding the problem of undesidered outcomes

A case is opened usually in May and should be closed by end of the year.

When this doesn't happen we have an undesidered outcome which could be:
- **Undesidered outcome 1**: The payment is late. A payment can be considered timely, if there has been a “begin payment” activity by the end of the year that was not eventually followed by “abort payment”. 
- **Undesidered outcome 2**:  The case needs to be reopened, either by the department (subprocess “Change”) or due to a legal objection by the applicant (subprocess “Objection”). This may result in additional payments or reimbursements (“payment_actual{x}“ > 0, where x ≥ 1 refers to the xth payment after the initial one) 

The business **question** given is the following:  
We would like to detect such cases as early as possible. Ideally, this should happen before a decision is made for this case (first occurrence of “Payment application+application+decide”). You may use data from previous years to make predictions for the current year

Also it would be interesting to analyze the differences beetwen years and the various departmens (as of the last two business questions)

## Process discovery and analysis

We will first perform process mining on the application payment logs to have an overview of the process and analyze it to get some insight about the task at hand.

The software we will be using is *Fluxicon Disco*

![pa1](../img/pa_1.png)

This is the payment application process of the entire log using 100% of activities ad paths setting set at 2%.
It is a little bit daunting and we don't need every detail of the process, we already know our business question so we need to filter it a little bit to have a clearer understanding.

Prior to simplifying the process we can take a look at some of the statistics of this log:

![me_1](../img/me_1.png)

This is the distribution of the events, the peaks we can observe are mainly from October to December of each year.

![me_2](../img/me_2.png)

This is the active cases graph, most of the cases start around May/June and end in December, but as we can see there are some residual events every year.

![me_3](../img/me_3.png)

Most of the cases require less than a year in total. The mean case duration is 43 weeks.
These are the total event log statistics:

![me_4](../img/me_4.png)

Before filtering the log to obtain an easier to grasp process map it could be useful to have at least an overview of the whole process.

![pa_3](../img/pa_3.png)

The red arrows point to the Payment application-application part of the map, that is the most frequent activities.
This is supported by setting the color schema to represent *Case frequency*, we can clearly see that all the darker coloured ones are part of this group (The darker the color the more frequent the activity)

![pa_4](../img/pa_4.png)

These are the activities regarding *Objections*, they are useful in the understanding of Undesidered outcome 2.

![pa_5](../img/pa_5.png)

These are the activities relative to the *Change* subprocess that are also useful in the study of Undesidered outcome 2. 
The remaining cells are relative to *Main* subprocess.

Now we can proceed with filtering, so let's recap what we absolutely need in our process map:
- Begin payment, for the study of undesidered outcome 1
- Abort payment, for the study of undesidered outcome 1
- Subprocess Change, for the study of undesidered outcome 2
- Subprocess Objection, for the study of undesidered outcome 2

Mantaining 100% of cases and filtering activities we don't need this seems like a good compromise of a process map for understanding.

![map](../img/pa_2.png)

Below every path frequency are included also the average duration of the activity to the next. The most notable is that from begin Payment to Change initialize there is an average long transition time.

## Tackling the problem

The first thing we need to do is to engineer the dataset of the payment applications to find out the labels of our undesidered outcomes.

In [2]:
df = pd.read_csv('../data/Payment application.csv')

Now that we loaded the data we can take a peek to think what we can do next

In [3]:
df.head()

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,...,(case) young farmer,activity,concept:name,docid,doctype,eventid,lifecycle:transition,note,subprocess,success
0,-18008611495569447,Payment application-Application-mail income,0;n/a,2015/05/08 00:00:00.000,Variant 8,8,960.35,NaN,NaN,NaN,...,False,mail income,mail income,-18008611495569447,Payment application,NaN,complete,none,Application,True
1,-18008611495569447,Payment application-Application-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 8,8,960.35,NaN,NaN,NaN,...,False,mail valid,mail valid,-18008611495569447,Payment application,NaN,complete,none,Application,True
2,-18008611495569447,Payment application-Application-initialize,Document processing automaton,2015/11/06 11:21:27.376,Variant 8,8,960.35,NaN,NaN,NaN,...,False,initialize,initialize,-18008611495569447,Payment application,-1.800861e+16,complete,none,Application,True
3,-18008611495569447,Payment application-Application-begin editing,727350,2015/11/10 17:17:50.045,Variant 8,8,960.35,NaN,NaN,NaN,...,False,begin editing,begin editing,-18008611495569447,Payment application,-1.080806e+17,complete,automatic,Application,True
4,-18008611495569447,Payment application-Application-calculate,727350,2015/11/10 17:17:57.085,Variant 8,8,960.35,NaN,NaN,NaN,...,False,calculate,calculate,-18008611495569447,Payment application,-1.080806e+17,complete,automatic,Application,True


## Undesidered outcomes 1

To find out our undesidered outcomes 1 we just need 3 activities for each case: Application initialize, Begin payment and abort payment.
A payment is late basically if the last activity is not a begin payment (So an abort or just initialized) or it is a begin payment but from a different year than the case year

In [4]:
uo1_code = """
activities = ['Payment application-Application-initialize', 'Payment application-Application-begin payment', 'Payment application-Application-abort payment']

df_prepared = df[df['Activity'].isin(activities)]

cases = df_prepared['Case ID'].unique()

uo1_list = []

for case in cases:
    is_late = False
    
    case_df = df_prepared[df_prepared['Case ID'] == case]
    last_activity = case_df.tail(1)
    
    if last_activity['Activity'].item() != 'Payment application-Application-begin payment':
        # Last activity is initialize or abort payment
        is_late = True
    else:
        payment_dt = datetime.strptime(last_activity['Complete Timestamp'].item(), '%Y/%m/%d %H:%M:%S.%f')
        case_year = last_activity['(case) year']
        if payment_dt.year > case_year.item():
            is_late = True
        
    uo1_list.append((case, is_late))
    
uo1_df = pd.DataFrame(uo1_list, columns=['Case ID', 'Late Payment'])

uo1_df.to_csv('../data/uo1.csv', index=False)
"""

In [5]:
uo1_df = pd.read_csv('../data/uo1.csv')

Then we can merge this with our dataset and perform the last tweakings to create a machine learning ready dataset

In [6]:
uo1_full_df = pd.merge(df , uo1_df, on='Case ID', how='left')
uo1_full_df.head()

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,...,activity,concept:name,docid,doctype,eventid,lifecycle:transition,note,subprocess,success,Late Payment
0,-18008611495569447,Payment application-Application-mail income,0;n/a,2015/05/08 00:00:00.000,Variant 8,8,960.35,NaN,NaN,NaN,...,mail income,mail income,-18008611495569447,Payment application,NaN,complete,none,Application,True,False
1,-18008611495569447,Payment application-Application-mail valid,0;n/a,2015/05/08 00:00:00.000,Variant 8,8,960.35,NaN,NaN,NaN,...,mail valid,mail valid,-18008611495569447,Payment application,NaN,complete,none,Application,True,False
2,-18008611495569447,Payment application-Application-initialize,Document processing automaton,2015/11/06 11:21:27.376,Variant 8,8,960.35,NaN,NaN,NaN,...,initialize,initialize,-18008611495569447,Payment application,-1.800861e+16,complete,none,Application,True,False
3,-18008611495569447,Payment application-Application-begin editing,727350,2015/11/10 17:17:50.045,Variant 8,8,960.35,NaN,NaN,NaN,...,begin editing,begin editing,-18008611495569447,Payment application,-1.080806e+17,complete,automatic,Application,True,False
4,-18008611495569447,Payment application-Application-calculate,727350,2015/11/10 17:17:57.085,Variant 8,8,960.35,NaN,NaN,NaN,...,calculate,calculate,-18008611495569447,Payment application,-1.080806e+17,complete,automatic,Application,True,False


Ok now we need to have a single row for each case (we will use just the case data for the prediction)

In [7]:
uo1_full_unique_df = uo1_full_df.drop_duplicates(subset='Case ID')
uo1_full_unique_df.head()

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) amount_applied0,(case) amount_applied1,(case) amount_applied2,(case) amount_applied3,...,activity,concept:name,docid,doctype,eventid,lifecycle:transition,note,subprocess,success,Late Payment
0,-18008611495569447,Payment application-Application-mail income,0;n/a,2015/05/08 00:00:00.000,Variant 8,8,960.35,NaN,NaN,NaN,...,mail income,mail income,-18008611495569447,Payment application,NaN,complete,none,Application,True,False
16,-18008611496121228,Payment application-Application-mail income,0;n/a,2015/05/08 00:00:00.000,Variant 43,43,28192.23,NaN,NaN,NaN,...,mail income,mail income,-18008611496121228,Payment application,NaN,complete,none,Application,True,False
37,-18008611495675691,Payment application-Application-mail income,0;n/a,2015/05/13 00:00:00.000,Variant 2,2,6684.43,NaN,NaN,NaN,...,mail income,mail income,-18008611495675691,Payment application,NaN,complete,none,Application,True,False
52,-18008611495740979,Payment application-Application-mail income,0;n/a,2015/04/10 00:00:00.000,Variant 7,7,6684.43,NaN,NaN,NaN,...,mail income,mail income,-18008611495740979,Payment application,NaN,complete,none,Application,True,False
71,-18008611496156872,Payment application-Application-mail income,0;n/a,2015/04/24 00:00:00.000,Variant 70,70,43403.12,NaN,NaN,NaN,...,mail income,mail income,-18008611496156872,Payment application,NaN,complete,none,Application,True,False


Now we need to get rid of the useless columns, without doubt we should get rid of the activity related columns using the reported ones in the BPI challenge.

In [8]:
uo1_full_unique_df.columns

Index(['Case ID', 'Activity', 'Resource', 'Complete Timestamp', 'Variant',
       'Variant index', '(case) amount_applied0', '(case) amount_applied1',
       '(case) amount_applied2', '(case) amount_applied3', '(case) applicant',
       '(case) application', '(case) area', '(case) basic payment',
       '(case) cross_compliance', '(case) department', '(case) greening',
       '(case) number_parcels', '(case) payment_actual0',
       '(case) payment_actual1', '(case) payment_actual2',
       '(case) payment_actual3', '(case) penalty_ABP', '(case) penalty_AGP',
       '(case) penalty_AJLP', '(case) penalty_AUVP', '(case) penalty_AVBP',
       '(case) penalty_AVGP', '(case) penalty_AVJLP', '(case) penalty_AVUVP',
       '(case) penalty_B16', '(case) penalty_B2', '(case) penalty_B3',
       '(case) penalty_B4', '(case) penalty_B5', '(case) penalty_B5F',
       '(case) penalty_B6', '(case) penalty_BGK', '(case) penalty_BGKV',
       '(case) penalty_BGP', '(case) penalty_C16', '(case) penalt

By taking a look at the BPI challenge document we can for sure drop values relative to the activity. Also we can drop values relative to the Variant as this won't be known prior to the prediction.

In [9]:
cols_to_drop = ['Activity', 'Complete Timestamp','Variant','Variant index', 'activity', 'concept:name', 'docid', 'doctype','eventid', 'lifecycle:transition','note','subprocess', 'success']

uo1_ready = uo1_full_unique_df.drop(cols_to_drop, axis='columns')

uo1_ready.columns

Index(['Case ID', 'Resource', '(case) amount_applied0',
       '(case) amount_applied1', '(case) amount_applied2',
       '(case) amount_applied3', '(case) applicant', '(case) application',
       '(case) area', '(case) basic payment', '(case) cross_compliance',
       '(case) department', '(case) greening', '(case) number_parcels',
       '(case) payment_actual0', '(case) payment_actual1',
       '(case) payment_actual2', '(case) payment_actual3',
       '(case) penalty_ABP', '(case) penalty_AGP', '(case) penalty_AJLP',
       '(case) penalty_AUVP', '(case) penalty_AVBP', '(case) penalty_AVGP',
       '(case) penalty_AVJLP', '(case) penalty_AVUVP', '(case) penalty_B16',
       '(case) penalty_B2', '(case) penalty_B3', '(case) penalty_B4',
       '(case) penalty_B5', '(case) penalty_B5F', '(case) penalty_B6',
       '(case) penalty_BGK', '(case) penalty_BGKV', '(case) penalty_BGP',
       '(case) penalty_C16', '(case) penalty_C4', '(case) penalty_C9',
       '(case) penalty_CC', '(case

Now we will save the file and proceed to use RapidMiner to create predictions

In [10]:
uo1_ready.to_csv('../data/uo1_ready.csv', index=False)

![ml_1](../img/ml_1.png)

We can see on a logarithmic scale that the dataset is heavy unbalanced, a simple approach would be to take all the true value and an equal (or multiplied by a little multiplier) number of random false values (based on how much data we have).

In [11]:
uo1_ready_t = uo1_ready[uo1_ready['Late Payment'] == True]
uo1_ready_f = uo1_ready[uo1_ready['Late Payment'] == False]

uo1_read_f_sampled = uo1_ready_f.sample(len(uo1_ready_t) * 2, random_state=1337)

uo1_balanced = pd.concat([uo1_read_f_sampled, uo1_ready_t])

uo1_balanced.to_csv('../data/uo1_ready_balanced.csv', index=False)

Now we are ready to proceed to create a machine learning model using RapidMiner.

Applying some basic data cleaning and feature selection we end up with the following results:
![ml_2](../img/ml_2.png)

The best model overall was Decision Tree and the model is the following:

![ml_3](../img/ml_3.png)

We can see that the best predictors for the undesidered outcome 1 are the payments, this is a useful insight for future applies.

## Undesidered outcome 2

To find undesidered outcome 2 we just need to check if the case has an objection or change activity.
Also we can't use as predictors the payment_actual columns as they are calculated after the undesidered outcome.
We also won't use 2017 data for training because we don't know if it will be changed or objected.

In [12]:
c2015 = df['(case) year'] == 2015
c2016 = df['(case) year'] == 2016
df_no2017 = df[c2015 | c2016]

After discarding year 2017 we can isolate cases that have objection or change just by leaving only those activity types and removing ID duplicates

In [13]:
subprocesses = ['Objection', 'Change']

filter_mask = df_no2017['subprocess'].isin(subprocesses)

df_objchg = df_no2017[filter_mask]

df_objchg_unique= df_objchg.drop_duplicates(subset='Case ID')

All these cases are reopened we set them to True

In [14]:
df_objchg_unique['Reopened'] = True
uo2_vals = df_objchg_unique[['Case ID', 'Reopened']]

C:\Users\miche\Anaconda3\envs\pmproject\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Then we join this dataset with the NO 2017 dataset and fill the missing values with False (they don't have objection or change)

In [15]:
uo2_df_raw = pd.merge(df_no2017, uo2_vals, on='Case ID', how='left')
uo2_df_raw['Reopened'] = uo2_df_raw['Reopened'].fillna(False)

uo2_df_unique = uo2_df_raw.drop_duplicates(subset='Case ID')
uo2_df_unique['Reopened'].value_counts()

False    24709
True      4593
Name: Reopened, dtype: int64

We can see that the dataset is unbalanced, in this case we will use the same number of data for both True and False.


In [16]:
uo2_t = uo2_df_unique[uo2_df_unique['Reopened'] == True]
uo2_f = uo2_df_unique[uo2_df_unique['Reopened'] == False]

uo2_f_sampled = uo2_f.sample(len(uo2_t), random_state=1337)

uo2_full_df = pd.concat([uo2_t, uo2_f_sampled])

uo2_full_df.to_csv('../data/uo2_full.csv', index=False)

Let's clean and create models using RapidMiner

These are the model results:

![ml2_1](../img/ml2_1.png)

And these are the weights of the Generalized Linear model (which performed the best)

![ml2_2](../img/ml2_2.png)